# Customizing Darknet's YOLOv4-tiny 

This notebook implements [YOLOv4 tiny](https://github.com/AlexeyAB/darknet/issues/6067) on your notebook for custom object detection.


We will take the following steps to implement YOLOv4 on our custom data:
* Configure our GPU environment on Google Colab
* Install the Darknet YOLOv4 training environment
* Download our custom dataset for YOLOv4 and set up directories
* Configure a custom YOLOv4 training config file for Darknet
* Train our custom YOLOv4 object detector
* Reload YOLOv4 trained weights and make inference on test images

In [ ]:
import os

from IPython.core.magic import register_line_cell_magic

# Step 1: Enable GPU within your notebook

Go to:
Edit -> Notebook Settings -> GPU


# Step 2: Cloning and Building Darknet

Clone darknet from AlexeyAB's famous repository, adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

In [ ]:
%cd /content/
%rm -rf darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

Following script enables you to build darknet so that you can then use the darknet executable file to run or train object detectors)

In [ ]:
!make

# Step 3: Download pre-trained weights for the convolutional layers.

This downloads weights for the convolutional layers of the YOLOv4-tiny network. By using these weights it helps your custom object detector to be way more accurate and not have to train as long.

In [ ]:
#download the newly released yolov4-tiny weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

# Step 4: Define Helper Function

Allows you to show the image in your Colab Notebook after running your detections

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib notebook

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# Step 5: Set up Custom Dataset

Use Roboflow to convert dataset from any format to the YOLO Darknet format. 

1. To do so, create a free [Roboflow account](https://app.roboflow.ai).
2. Upload your images and their annotations (in any format: VOC XML, COCO JSON, TensorFlow CSV, etc).
3. Apply preprocessing and augmentation steps you may like. Recommend at least `auto-orient` and a `resize` to 416x416. Generate your dataset.
4. Export your dataset in the **YOLO Darknet format**.
5. Copy your download link, and paste it below. (do not share this link)

See Roboflow's [blog post](https://blog.roboflow.ai/training-yolov4-on-a-custom-dataset/) for greater detail.

In [ ]:
!curl -L "PASTE_LINK_HERE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Set up training file directories for custom dataset

In [ ]:
%cp train/_darknet.labels data/obj.names
%mkdir data/obj

Copy images and lables to required directory `./data/obj` and create the `obj.data` file

In [ ]:
%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 2\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

Write the image list for train.txt

In [ ]:
with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

Write the image list for valid.txt

In [ ]:
with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

## Step 5.1 Edit .cfg file

The .cfg file needs to fit the needs of the custom object detector.

### Variable Configuration (according to Darknet)

- **batch** = 64

- **subdivisions** = 16 *(32 or even 64 if you run into CUDA memory issues)*

- **width** and **height** = 416 *(these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)*

- **max_batches** = (# of classes) * 2000  *(but no less than 6000 so if you are training for 1, 2, or 3 classes it will be 6000)*

- **steps** = (80% of max_batches), (90% of max_batches)

- **filters** = (# of classes + 5) * 3

**Optional:** If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to **random = 0** to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.



We build the config file dynamically based on the class numbers

In [ ]:
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_darknet.labels')
max_batches = num_classes*2000
steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(steps1)+','+str(steps2)
num_filters = (num_classes + 5) * 3

Remove existing config file and customize iPython writefile so we can write variables

In [ ]:
if os.path.exists('./cfg/custom-yolov4-tiny-detector.cfg'): os.remove('./cfg/custom-yolov4-tiny-detector.cfg')

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Create the yolov4-tiny-obj.cfg file

In [ ]:
%%writetemplate ./cfg/yolov4-tiny-obj.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches = {max_batches}
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

# Step 6: Train the Custom Object Detector
```
!./darknet detector train <path to obj.data> <path to custom config> yolov4.conv.29 -dont_show -map
```

- **dont_show** : stops chart from popping up since Colab Notebook can't open images on the spot
- **map** : overlays mean average precision on chart to see model accuracy, only add map flag if you have a validation dataset)

uncomment %%capture below if you run into memory issues or your Colab is crashing

In [ ]:
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

## Step 6.1 (optional): Leaving Colab Idle Issue Fix 

The training might take a while (sometimes even 20 hours). However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
This clicks the screen every 10 minutes so that you don't get kicked off for being idle

## Step 6.2 (optional): Continue Training using saved weights

If you get an error or Colab goes idle during training, use your partially trained model and weights to continue training.

Every 100 iterations a weights file called **yolov4-obj_last.weights** is saved to **./backup/** folder and every 1000 iterations a **yolov4-obj_{x}000.weights** is created.  This however requires you to keep an eye on newly created weights and download them for later.

You can change the backup folder to your drive if needed (**check Step 5.1**).

Run the following command
```
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg ./backup/yolov4-obj_last.weights -dont_show
```

In [ ]:
# !./darknet detector train data/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_last.weights -dont_show
# or
# !./darknet detector train data/obj.data cfg/yolov4-obj.cfg backup/yolov4-obj_{x}000.weights -dont_show

## Step 6.3 (optional): Iteration vs Loss Chart

After training, you can observe a chart of how your model did throughout the training process by running the below command. It shows a chart of your average loss vs. iterations. For your model to be 'accurate' you should aim for a loss under 2.

In [ ]:
imShow('chart.png')

# Step 7: Run the Object Detector


The tiny model has it's labels hardcoded in coco.names file, therefore copy the obj.names content to the coco.names file.

In [ ]:
%cp data/obj.names data/coco.names

Choose a random test image for the detector

In [ ]:
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
img_path = "test/" + random.choice(test_images);

Once the models has completed training or the average loss hasn't changes as much, a **yolov4-obj_best.weights** file is created in backup. We use this to test our model.

Run the detector and show its predictions
```
!./darknet detect <path to obj.cfg> <path to obj.weights> <image path>  -dont-show
```

In [ ]:
!./darknet detect cfg/yolov4-tiny-obj.cfg backup/yolov4-tiny-obj_best.weights {img_path} -dont-show
imShow('predictions.jpg')

## Step 7.1 (alternative): Run Detector on Video Files

The detector can also be used on video files, use the following command to run the detector:

```
!./darknet detector demo <path to obj.data> <path to obj.cfg> <path to obj.weights> -dont_show <path to video> -i 0 -out_filename <path to output file>
```

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-tiny-obj.cfg backup/yolov4-tiny-obj_best.weights -dont_show /content/test.mp4 -i 0 -out_filename results.avi

# Done!

